In [101]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [102]:
file_path = os.path.join(os.getcwd(), '..', 'Data', 'Recipes_Clean.csv')
df = pd.read_csv(file_path)

In [111]:
df.sample(1)

,Page No,Recipe Name,Cuisine,Course,Diet,Prep Time,Cook Time,Total Time,Servings,Ingredient Names,Equipments,Average Rating,Total Ratings,Image URL,Recipe URL,combined_features
3988,172,Punjabi Sabut Moong Ki Dal Recipe,North Indian,Lunch,High Protein Vegetarian,10,30,40,2,हरी मूंग दाल|प्याज|टमाटर|अदरक|लहसुन|हरी मिर्च|...,"Hard Anodized Pressure Cooker,Tadka Pan (Seaso...",5.0,840,https://www.archanaskitchen.com/imgcache/image...,https://www.archanaskitchen.com/punjabi-sabut-...,Punjabi Sabut Moong Ki Dal Recipe North Indian...


### Selecting features for creating tfidf vectors and calculating cosine similarity

In [104]:
features = ['Recipe Name', 'Cuisine', 'Course', 'Diet', 'Prep Time', 'Cook Time', 'Total Time', 'Servings', 'Ingredient Names', 'Equipments']
df['combined_features'] = df[features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [105]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

### Fetching recommendations based on cosine simlarity

In [106]:
def get_recommendations(recipe_name, n_recommendations=10, cosine_sim=cosine_sim):
    idx = df[df['Recipe Name'] == recipe_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n_recommendations+1] 
    recipe_indices = [i[0] for i in sim_scores]
    return df['Recipe Name'].iloc[recipe_indices]

In [107]:
recipe_name = 'Roasted Cauliflower Dill Soup Recipe'
recommended_recipes = get_recommendations(recipe_name, 5)
print(recommended_recipes)

1753                      Roasted Cauliflower Soup Recipe
1989            Spiced Cauliflower And Almond Soup Recipe
5038       Penne Pasta Recipe In Creamy Cauliflower Sauce
4559    Aloo Gobi Sabzi Recipe - Spicy Potato Cauliflo...
745     Karnataka Style Bassaru Recipe - Dill flavored...
Name: Recipe Name, dtype: object


### Saving Model

In [108]:
save_df = df[['Recipe Name', 'Image URL', 'Recipe URL']]
model_save_path = os.path.join(os.getcwd(), '..', 'Model', 'model_pickle.pkl')
with open(model_save_path, "wb") as f:
    pickle.dump((save_df, tfidf_matrix), f)